In [ ]:
!pip install duckduckgo-search==5.1.0 langchain langchain-groq wikipedia gradio torch torchvision open_clip_torch clip-interrogator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 787.8/787.8 kB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import re
import os
import requests
from PIL import Image
import gradio as gr
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.tools import DuckDuckGoSearchRun
from langchain.utilities import WikipediaAPIWrapper
from clip_interrogator import Config, Interrogator
from langchain.agents import Tool, initialize_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.llm import LLMChain
from google.colab import userdata
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
GROQ_API_KEY=GROQ_API_KEY

In [ ]:
caption_model_name = 'blip-large'
clip_model_name = 'ViT-L-14/openai'

config = Config()
config.clip_model_name = clip_model_name
config.caption_model_name = caption_model_name
ci = Interrogator(config)

llm = ChatGroq(temperature=0.5,
               groq_api_key=GROQ_API_KEY,
               model_name="mixtral-8x7b-32768",
               max_tokens=20000)

Loading caption model blip-large...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Loading CLIP model ViT-L-14/openai...


100%|███████████████████████████████████████| 933M/933M [00:40<00:00, 23.1MiB/s]
ViT-L-14_openai_artists.safetensors: 100%|██████████| 16.2M/16.2M [00:00<00:00, 93.0MB/s]
ViT-L-14_openai_flavors.safetensors: 100%|██████████| 155M/155M [00:00<00:00, 197MB/s]
ViT-L-14_openai_mediums.safetensors: 100%|██████████| 146k/146k [00:00<00:00, 15.5MB/s]
ViT-L-14_openai_movements.safetensors: 100%|██████████| 307k/307k [00:00<00:00, 10.3MB/s]
ViT-L-14_openai_trendings.safetensors: 100%|██████████| 111k/111k [00:00<00:00, 10.1MB/s]
ViT-L-14_openai_negative.safetensors: 100%|██████████| 63.2k/63.2k [00:00<00:00, 11.4MB/s]


Loaded CLIP model and data in 55.05 seconds.


In [ ]:
def image_to_prompt(image):
    ci.config.chunk_size = 2048 if ci.config.clip_model_name == "ViT-L-14/openai" else 1024
    ci.config.flavor_intermediate_count = 2048 if ci.config.clip_model_name == "ViT-L-14/openai" else 1024
    image = image.convert('RGB')
    mode="best"
    if mode == 'best':
        return ci.interrogate(image)


In [ ]:
wikipedia = WikipediaAPIWrapper()
search = DuckDuckGoSearchRun()

wikipedia_tool = Tool(
    name='wikipedia',
    func= wikipedia.run,
    description="Useful for when you need to look up a topic, country or person on wikipedia is the best website for fact check and any other details on any subject."
)

duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func= search.run,
    description="Useful for when you need to do a search on the internet to find information that another tool can't find. Always be specific with your input."
)

tools = [
    Tool(
        name = "DuckDuckGo Search",
        func=duckduckgo_tool.run,
        description="useful for when you need answer questions from internet"
    )
]

tools.append(wikipedia_tool)

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10,
)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
def menu_prompt(title):
    menu_output = """
        EXAMPLE OUTPUT: Return a dictionary with the following keys:
        "ingredients": Comma separated ingredients in a list
        "description": description about the Dish
        "allergens": Comma separated allergens in a list
        "additional_information": Comma separated additional_information in a list
    """

    prompt_menu = f'''
    <s>[INST]As a restaurant menu manager, your role is to gather essential details about a dish based on its input <{title}>(Name of the dish).
    Based on the Name <{title}> of the dish.Please provide the following information:
    1. Ingredients: List all ingredients included in the dish.
    2. Description: Briefly describe the dish.
    3. Allergens: Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
    4. Additional Information: Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].
    OUTPUT: Provide the extracted information for each label as a JSON object.
    ''' + menu_output + '''
    [/INST]</s>'''
    return prompt_menu

In [ ]:
def social_media_prompt(title):

    social_output = """
        EXAMPLE OUTPUT: Return the three posts in a dictionary :
        Description of the first post
        Description of the second post
        Description of the third post
    """

    prompt_social_media = f'''
    <s>[INST]As a social media content manager, your task is to create three different
    promotional post for Facebook/Instagram based on the given input <{title}> and its description.

    These posts should showcase irresistible <{title}>,relevant options from this list[promotions , Delivery , Notoriety,Reservations,Theme Days, Menu] embodying a tone inspired.
    Ensure the post includes relevant <content,hashtags,emoji's,persuasive call-to-action> tailored specifically for restaurants.

    OUTPUT: Three posts must be in valid dictionary. Don’t add an explanation beyond the dictionary.
    ''' + social_output + '''[/INST]</s>'''
    return prompt_social_media

In [ ]:
def get_router(image,title):
    if image:
        image_description = image_to_prompt(image)
    else:
        image_description = ""
    title_concate=title+image_description
    prompt_menu=menu_prompt(title_concate)
    prompt_social_media=social_media_prompt(title_concate)

    prompt_infos = [
      {
      "name": "Social Media Manager",
      "description": "Good for answering questions about goals[promotions , Delivery , Notoriety,Reservations,Theme Days, Menu] Accepts Input image and goals can be any one of them [promotions , Delivery , Notoriety,Reservations,Theme Days, Menu].Creatig three posts on Instagram/Facebook",
      "prompt_template": prompt_social_media,
      },
      {
          "name": "Menu Manager",
          "description": "Good for answering questions about Menu (dishes)",
          "prompt_template": prompt_menu,
      },]
    # map destination chains
    destination_chains = {}
    for prompt_info in prompt_infos:
        name = prompt_info["name"]
        prompt_template = prompt_info["prompt_template"]
        prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
        chain = LLMChain(llm=llm, prompt=prompt)
        destination_chains[name] = chain
    default_chain = ConversationChain(llm=llm, output_key="text")

    # Creating LLMRouterChain
    destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
    destinations_str = "\n".join(destinations)
    router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
    router_prompt = PromptTemplate(
        template=router_template,
        input_variables=["input"],
        output_parser=RouterOutputParser(),
    )

    # creating the router chain
    router_chain = LLMRouterChain.from_llm(llm, router_prompt)

    # Multiple Prompt Chain
    chain = MultiPromptChain(
        router_chain=router_chain,
        destination_chains=destination_chains,
        default_chain=default_chain,
        verbose=True,
    )
    # Get response from the agent
    response = zero_shot_agent.run(chain.run(title_concate))
    return response


In [ ]:

image_path="/content/pizza.jpeg"
image = Image.open(image_path)
title="Promotions"
response=get_router(image,title)
print(response)


100%|██████████| 50/50 [00:00<00:00, 180.13it/s]




> Entering new MultiPromptChain chain...
Social Media Manager: {'input': "Promotions\nthere is a pizza with several different toppings on a wooden board, profile picture 1024px, smooth oval shape face, denoised, viewed in profile, worth1000.com, advertising photography, creative commons attribution, primary colors are white, bird's eye view, steaming food on the stove, canva, an ai generated image"}
> Finished chain.


> Entering new AgentExecutor chain...
The question doesn't seem to require any external research, as the input is a dictionary containing promotional posts. I can analyze the provided information to answer potential questions.

Question: Analyze the promotional posts and identify the unique selling points for each post.
Thought: I will go through each post and identify the unique selling points by examining the content, hashtags, emojis, and call-to-action.

Post1:
- Content: Mentions irresistible pizza options, speedy delivery service.
- Hashtags: #pizzalove, #foodie,

In [ ]:
'''
title = "Interactive demo: Azzu - CopyWriterAgent"
description = ""

examples = [["example01.jpg","Lasagna"],["tiramisu.jpg","Tiramisu"],["pizza.jpg","Pizza Margherita"],["example02.jpg","Pasta"],
           ["socialmedia.jpg","Promotions"]]

# Create Gradio interface
iface = gr.Interface(get_router,
                     inputs=[
                         gr.Image(type='pil'),
                         "text"
                     ],
                     outputs="text",
                     title=title,
                     #examples=examples,
                     description=description)
iface.launch(inline=True, debug=True)
'''